In [ ]:

import json
import websocket
import datetime
import numpy as np
import pandas as pd
from datetime import datetime, timezone
import time
import joblib
import os
import sys


In [ ]:
def my_json(json_message):
    size = len(np.array(json_message['bids'])[:,0])    
    return {
        **{'ts':[time.mktime(datetime.now().timetuple())]},
        **{'bp'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['bids'])[:,0])},
        **{'bs'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['bids'])[:,1])},
        **{'ap'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['asks'])[:,0])},
        **{'as'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['asks'])[:,1])}}
    
def my_json_0(size):    
    return {
        **{'ts':[time.mktime(datetime.now().timetuple())]},
        **{'bp'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
        **{'bs'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
        **{'ap'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
        **{'as'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)}}
        

        

In [ ]:
def my_json(json_message):
    size = len(np.array(json_message['bids'])[:,0])    
    return {
        **{'primary_key':[datetime.now()]},
        **{'bp'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['bids'])[:,0])},
        **{'bs'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['bids'])[:,1])},
        **{'ap'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['asks'])[:,0])},
        **{'as'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['asks'])[:,1])}}
    
def my_json_0(size):    
    return {
        **{'primary_key':[datetime.now()]},
        **{'bp'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
        **{'bs'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
        **{'ap'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
        **{'as'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)}}

In [ ]:

df_0 = pd.DataFrame.from_dict(my_json_0(5))

def ws_trades(df_0): 
    symbol = 'BTCUSDT'
    depth = 5
    socket = 'wss://stream.binance.com:9443/ws/{}@depth{}'.format(symbol.lower(),
                                                                depth)
    
    
    def on_message(wsapp,message):  
        json_message = json.loads(message)
        handle_trades(json_message)

    def on_error(wsapp,error):
        print(error)

    wsapp = websocket.WebSocketApp(socket, on_message=on_message, on_error=on_error)
    wsapp.run_forever()
    
def handle_trades(json_message):    
    df = pd.DataFrame.from_dict(my_json(json_message))
    df_0 = df_0.append(df, ignore_index = True)
    
    print(df_0)
    
    
    
    print("-----------------------")

In [ ]:
class StreamingData():
    def __init__(self):
        self.depth = 5     
        self.df_0 = None       
        self.model = joblib.load('/Users/fipm/code/abefarkas/Thalassa_Regime_Classifier/model.joblib')    
        self.symbol = 'BTCUSDT'        
        self.socket = 'wss://stream.binance.com:9443/ws/{}@depth{}'.format(self.symbol.lower(),self.depth)
        
    def handle_trades(self,json_message):    
        df = pd.DataFrame.from_dict(self.my_json(json_message))
        self.df_0 = pd.concat([self.df_0, df], axis=0)   
        df_clean = self.my_return().dropna()
        # if len(df_clean)>=1:
        #     print(self.model.predict(df_clean))
        #     send to database 
        return self.model.predict(df_clean)
        
    def my_return(self):
        return self.preprocessing_streamed_data(self.df_0, rolling_window=2)

    def on_message(self,wsapp,message):  
        json_message = json.loads(message)
        self.handle_trades(json_message)

    def on_error(self,wsapp,error):
        print(error)

    def start(self):        
        self.df_0 = pd.DataFrame.from_dict(self.my_json_0(self.depth))
        wsapp = websocket.WebSocketApp(self.socket, on_message=self.on_message, on_error=self.on_error)
        wsapp.run_forever()
        
    def preprocessing_streamed_data(self,df_ob, rolling_window=2):
        '''preprocessing of data for streamed data'''                
        
        # aggregating by seconds     
        df_agg = df_ob.groupby(pd.Grouper(key='primary_key', axis=0, freq='S')).mean()
        # applying rolling window of rolling_window lenght
        df_agg = df_agg.rolling(str(rolling_window)+'S').mean()
        # moving the index as a column
        df_agg.reset_index(inplace=True)

        return df_agg
        
    def my_json(self, json_message):
        size = len(np.array(json_message['bids'])[:,0])    
        return {
            **{'primary_key':[datetime.now()]},
            **{'bp'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['bids'])[:,0])},
            **{'bs'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['bids'])[:,1])},
            **{'ap'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['asks'])[:,0])},
            **{'as'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['asks'])[:,1])}}
    
    def my_json_0(self, size):    
        return {
            **{'primary_key':[datetime.now()]},
            **{'bp'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
            **{'bs'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
            **{'ap'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
            **{'as'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)}}

        

    

In [ ]:
from websocket import create_connection
import json

class StreamingData2():
    def __init__(self):
        self.depth = 5     
        self.df_0 = None       
        # self.model = joblib.load('/Users/fipm/code/abefarkas/Thalassa_Regime_Classifier/model.joblib')    
        self.symbol = 'BTCUSDT'        
        self.socket = 'wss://stream.binance.com:9443/ws/{}@depth{}'.format(self.symbol.lower(),self.depth)
        self.ws = None
        
    def handle_trades(self):    
                
        df = pd.DataFrame.from_dict(self.my_json(json.loads(self.ws.recv())))
        self.df_0 = pd.concat([self.df_0, df], axis=0)   
        # df_clean = self.my_return().dropna()
        # if len(df_clean)>=1:
        #     print(self.model.predict(df_clean))
        #     send to database 
        # return self.model.predict(df_clean)
        
    def my_return(self):
        return self.preprocessing_streamed_data(self.df_0, rolling_window=2)

    def start(self):        
        self.df_0 = pd.DataFrame.from_dict(self.my_json_0(self.depth))
        self.ws = create_connection(self.socket)
        self.handle_trades()
        
        
    def preprocessing_streamed_data(self,df_ob, rolling_window=2):
        '''preprocessing of data for streamed data'''                
        
        # aggregating by seconds     
        df_agg = df_ob.groupby(pd.Grouper(key='primary_key', axis=0, freq='S')).mean()
        # applying rolling window of rolling_window lenght
        df_agg = df_agg.rolling(str(rolling_window)+'S').mean()
        # moving the index as a column
        df_agg.reset_index(inplace=True)

        return df_agg
        
    def my_json(self, json_message):
        
        size = len(np.array(json_message['bids'])[:,0])    
        return {
            **{'primary_key':[datetime.now()]},
            **{'bp'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['bids'])[:,0])},
            **{'bs'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['bids'])[:,1])},
            **{'ap'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['asks'])[:,0])},
            **{'as'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['asks'])[:,1])}}
    
    def my_json_0(self, size):    
        return {
            **{'primary_key':[datetime.now()]},
            **{'bp'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
            **{'bs'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
            **{'ap'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
            **{'as'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)}}

        


In [11]:
w = StreamingData2()
w.start()
while True:
    w.handle_trades()
    print(w.my_return())

          primary_key       bp1       bp2       bp3       bp4        bp5  \
0 2022-05-26 15:43:25       NaN       NaN       NaN       NaN        NaN   
1 2022-05-26 15:43:26       NaN       NaN       NaN       NaN        NaN   
2 2022-05-26 15:43:27  29599.25  29597.70  29597.47  29595.91  29595.590   
3 2022-05-26 15:43:28  29595.12  29594.33  29594.21  29593.09  29592.915   

        bs1      bs2       bs3      bs4  ...       ap1       ap2        ap3  \
0       NaN      NaN       NaN      NaN  ...       NaN       NaN        NaN   
1       NaN      NaN       NaN      NaN  ...       NaN       NaN        NaN   
2  0.001890  0.00067  0.169090  0.17441  ...  29599.26  29599.96  29599.990   
3  0.482595  0.02258  0.091025  0.17441  ...  29595.13  29596.52  29596.585   

        ap4       ap5      as1       as2    as3       as4      as5  
0       NaN       NaN      NaN       NaN    NaN       NaN      NaN  
1       NaN       NaN      NaN       NaN    NaN       NaN      NaN  
2  29600.00  296

KeyboardInterrupt: 

In [ ]:
# Import WebSocket client library (and others)
import websocket
import _thread
import time

# Define WebSocket callback functions
def ws_message(ws, message):
    # print("WebSocket thread: %s" % message)
    print('di')

def ws_open(ws):
    ws.send('{"event":"subscribe", "subscription":{"name":"trade"}, "pair":["XBT/USD","XRP/USD"]}')

def ws_thread(*args):
    ws = websocket.WebSocketApp("wss://ws.kraken.com/", on_open = ws_open, on_message = ws_message)
    ws.run_forever()

# Start a new thread for the WebSocket interface
_thread.start_new_thread(ws_thread, ())

# Continue other (non WebSocket) tasks in the main thread
while True:
    time.sleep(5)
    print("Main thread: %d" % time.time())

In [ ]:
# Import WebSocket client library
from websocket import create_connection

# Connect to WebSocket API and subscribe to trade feed for XBT/USD and XRP/USD

ws = create_connection("wss://ws.kraken.com/")
ws.send('{"event":"subscribe", "subscription":{"name":"trade"}, "pair":["XBT/USD","XRP/USD"]}')

# Infinite loop waiting for WebSocket data
while True:
    print(ws.recv())
    
    #     // URL connection
    # const accountAndOrderAndTransfers = new WebSocket("wss://dex.binance.org/api/ws/bnb1m4m9etgf3ca5wpgkqe5nr6r33a4ynxfln3yz4v");

    # // Or Subscribe method
    # const conn = new WebSocket("wss://dex.binance.org/api/ws");
    # conn.onopen = function(evt) {
    #     conn.send(JSON.stringify({ method: "subscribe", topic: "orders", address: "bnb1m4m9etgf3ca5wpgkqe5nr6r33a4ynxfln3yz4v" }));
    # }

In [ ]:
# Import WebSocket client library
from websocket import create_connection

depth = 5     
symbol = 'BTCUSDT'        
socket = 'wss://stream.binance.com:9443/ws/{}@depth{}'.format(symbol.lower(),depth)
# Connect to WebSocket API and subscribe to trade feed for XBT/USD and XRP/USD
ws = create_connection(socket)
# Infinite loop waiting for WebSocket data
while True:
    print(ws.recv())

In [ ]:
  
# initialising dictionaries
ini_dictionary1 = {'nikhil': 1, 'akash' : 5,
                     'manjeet' : 10, 'akshat' : 15}
ini_dictionary2 = {'akash' : 7, 'akshat' : 5,
                                          'm' : 15}

ini_dictionary1.update(ini_dictionary2)
ini_dictionary1  


In [ ]:
json_message ={'lastUpdateId': 19493390097, 
               'bids': [['29724.70000000', '8.90412000'], ['29747.650000', '0.00429000'], ['29724.62000000', '0.08293000'], ['29724.61000000', '0.01998000'], ['29724.57000000', '0.16835000']], 
               'asks': [['29724.71000000', '0.00001000'], ['29724.72000000', '0.00075000'], ['29724.73000000', '0.01237000'], ['29724.75000000', '0.01237000'], ['29724.77000000', '0.01000000']]}


{'bp'+str(key):value for key,value in zip(np.arange(0,5),np.array(json_message['bids'])[:,0]) }
{'bs'+str(key):value for key,value in zip(np.arange(0,5),np.array(json_message['bids'])[:,1]) }
{'bp'+str(key):value for key,value in zip(np.arange(0,5),np.array(json_message['bids'])[:,0]) }
{'bp'+str(key):value for key,value in zip(np.arange(0,5),np.array(json_message['bids'])[:,0]) }


In [ ]:
json_message ={'lastUpdateId': 19493390097, 
               'bids': [['29724.70000000', '8.90412000'], ['nan', '0.00429000'], ['29724.62000000', '0.08293000'], ['29724.61000000', '0.01998000'], ['29724.57000000', '0.16835000']], 
               'asks': [['29724.71000000', '0.00001000'], ['29724.72000000', '0.00075000'], ['29724.73000000', '0.01237000'], ['29724.75000000', '0.01237000'], ['29724.77000000', '0.01000000']]}
aaa(bp1) bid price 1 = 29724.70000000
(bs1) bid size 1 = 8.90412000

(ap1) asd price 1 = 29724.71000000
(as1) ask size 1 = 0000100

{'ts':json_message['lastUpdateId'],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],  
  'bp1':json_message['bids'][0][0],
  }

symbol,ts,last_update_id,bp1,bs1,bp2,bs2,bp3,bs3,bp4,bs4,bp5,bs5,bp6,bs6,bp7,bs7,bp8,bs8,bp9,bs9,bp10,bs10,bp11,bs11,bp12,bs12,bp13,bs13,bp14,bs14,bp15,bs15,bp16,bs16,bp17,bs17,bp18,bs18,bp19,bs19,bp20,bs20,ap1,as1,ap2,as2,ap3,as3,ap4,as4,ap5,as5,ap6,as6,ap7,as7,ap8,as8,ap9,as9,ap10,as10,ap11,as11,ap12,as12,ap13,as13,ap14,as14,ap15,as15,ap16,as16,ap17,as17,ap18,as18,ap19,as19,ap20,as20

In [ ]:
import pyspark
from pyspark.streaming import StreamingContext

In [ ]:
#Uncomment the next line to run the code block on jupyter. Keep it commented if copy-pasting into the pyspark shell
sc = pyspark.SparkContext()

# This tells Spark Streaming to bacth-up the contents of a data stream and "ingest" them every 10 seconds.
ssc = StreamingContext(sc,10)

# Tell spark to listen on port 9999 of our localhost.
lines = ssc.socketTextStream('wss://stream.binance.com/ws/BTCUSDT@depth5', 9443)
lines = ssc.socketTextStream('localhost', 9999)    

words = lines.flatMap(lambda line : line.split(" "))

pairs = words.map(lambda word: (word, 1))
wordCount = pairs.reduceByKey(lambda a, b: a + b)

wordCount.pprint()

ssc.start()
ssc.awaitTermination()

In [ ]:
#Uncomment the next line to run the code block on jupyter. Keep it commented if copy-pasting into the pyspark shell
sc = pyspark.SparkContext()

# This tells Spark Streaming to bacth-up the contents of a data stream and "ingest" them every 10 seconds.
ssc = StreamingContext(sc,10)

# Tell spark to listen on port 9999 of our localhost.
lines = ssc.socketTextStream('wss://stream.binance.com/ws/BTCUSDT@depth5', 9443)



lines.pprint()

ssc.start()
ssc.awaitTermination()

In [ ]:
ssc.stop()